In [1]:
#Bring in Needed Packages
import pandas as pd

In [3]:
#Load in dataset
data = r"InputFilePath.csv"
#Skip any bad lines
df = pd.read_csv(data, on_bad_lines='skip')

In [4]:
#Drop NaN returns
df = df.dropna(subset=[df.columns[0]])
#Create new column headers
new_header = df.iloc[1]
df = df[1:]
df.columns = new_header
df= df.drop(df.index[0])
#Remove zeros from Count column
df = df[df["Count"] != 0]

In [5]:
#Change from CSV into Excel
df.to_excel("output.xlsx", sheet_name="All_Data", index=False)

#Filter on New Patients and Consults, remove duplicates, create new sheet for data
filtered = df[df["Department"].isin(["New Patient Visits", "Consults"])]
filtered = filtered.drop_duplicates(subset="Encounter")
with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer: filtered.to_excel(writer, sheet_name="New Patients", index=False)
    
#Filter on Established Patients, remove duplicates, create new sheet for data
filtered = df[df["Department"].isin(["Established Visits"])]
filtered = filtered.drop_duplicates(subset="Encounter")
with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer: filtered.to_excel(writer, sheet_name="Established", index=False)
    
#Filter on Post Op, remove duplicates, create new sheet for data
filtered = df[df["Department"].isin(["Post Op"])]
filtered = filtered.drop_duplicates(subset="Encounter")
with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer: filtered.to_excel(writer, sheet_name="Post Op", index=False)
    
#Filter on Injections and Lasers, remove duplicates, create new sheet for data
filtered = df[df["Department"].isin(["Injections", "Lasers"])]
filtered = filtered.drop_duplicates(subset="Encounter")
with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer: filtered.to_excel(writer, sheet_name="DTI", index=False)

In [6]:
#Combine New Patient, Established, Post Op sheets into one combined sheet
new_patients = pd.read_excel("output.xlsx", sheet_name="New Patients") 
established = pd.read_excel("output.xlsx", sheet_name="Established") 
post_op = pd.read_excel("output.xlsx", sheet_name="Post Op")

combined = pd.concat([new_patients, established, post_op], ignore_index=True)

with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer: combined.to_excel(writer, sheet_name="Combined", index=False)

In [7]:
#Compare encounter numbers between Combined sheet and DTI sheet
combined = pd.read_excel("output.xlsx", sheet_name="Combined") 
dti = pd.read_excel("output.xlsx", sheet_name="DTI")
shared_mask = dti["Encounter"].isin(combined["Encounter"])
dti.loc[shared_mask, "Department"] = "Visit Encounter Associated"
with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer: dti.to_excel(writer, sheet_name="DTI_Updated", index=False)

In [8]:
#Combine the data from the Combined sheet and the DTI sheet
dti = pd.read_excel("output.xlsx", sheet_name="DTI_Updated")
extra = dti[dti["Department"].isin(["Injections", "Lasers"])]
final_combined = pd.concat([combined, extra], ignore_index=True)
with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl") as writer: final_combined.to_excel(writer, sheet_name="Combined_Final", index=False)